# CH01- Quick Start

[pytorch tutorial link](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html)



## 1-处理数据

PyTorch 有两个用于处理数据的函数： 

+ torch.utils.data.DataLoader

+ torch.utils.data.Dataset

其中，Dataset 存储样本及相应标签；DataLoader一次加载一批量Dataset中的数据。

`torchvision.datasets`模块包含许多现有的数据集，如 CIFAR、COCO、FashionMNIST等数据集。

`torchvision.dataset`包含两个参数：transform和 target_transform，分别表示修改样本和修改标签。

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets,transforms

# ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

### 1.1 下载数据集

In [2]:
# Download training data from open datasets.
download_flag=False

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    # download=True,
    download=download_flag,
    transform=transforms.ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    # download=True,
    download=download_flag,
    transform=transforms.ToTensor(),
)

100.0%
Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw

100.6%
Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw

100.0%
Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw

119.3%
D:\Application\Anaconda\envs\pytorchGPU\lib\site-packages\torchvision\datasets\mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)
Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw

Processing...
Done!

### 1.2 小批量加载数据

将Dataset作为参数传递给DataLoader。这在我们的数据集上包装了一个迭代，并支持自动批处理、采样、混洗和多进程数据加载。

这里我们定义了 batch_size=64 的批量大小，即数据加载器的每一次迭代将返回 64 个特征和标签组成的批量。

In [3]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


## 2-创建模型

在 PyTorch 中定义神经网络，需要创建一个继承自`nn.Module`的类:

+ 在__init__函数中定义网络的各个层，

+ 在`forward()`函数中指定，在网络中，数据如何前向传播

GPU可以加速神经网络中的数据计算（如果可用）。

In [4]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


## 3-优化模型参数

为了训练上述定义的模型，需要定义损失函数和优化器

In [5]:

loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

在一次训练迭代中，模型对小批量训练集进行预测，然后计算出预测误差，再对预测误差做反向传播，以调整模型参数。

In [6]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

除了定义训练函数之外，还需要定义一个测试函数，以检查模型在测试集上的性能。

In [7]:
def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

设定epoch=5,在每个epoch结束后，打印出模型的准确度和损失。

随着epoch增加，loss越小越好，准确度越高越好。

In [8]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model)

print("Done!")

Epoch 1
-------------------------------
loss: 2.306628  [    0/60000]
loss: 2.298265  [ 6400/60000]
loss: 2.280692  [12800/60000]
loss: 2.277453  [19200/60000]
loss: 2.260351  [25600/60000]
loss: 2.254862  [32000/60000]
loss: 2.235303  [38400/60000]
loss: 2.215355  [44800/60000]
loss: 2.207268  [51200/60000]
loss: 2.211222  [57600/60000]
Test Error: 
 Accuracy: 52.2%, Avg loss: 0.034546 

Epoch 2
-------------------------------
loss: 2.186224  [    0/60000]
loss: 2.201916  [ 6400/60000]
loss: 2.158741  [12800/60000]
loss: 2.186998  [19200/60000]
loss: 2.128443  [25600/60000]
loss: 2.124032  [32000/60000]
loss: 2.089390  [38400/60000]
loss: 2.047239  [44800/60000]
loss: 2.055050  [51200/60000]
loss: 2.060381  [57600/60000]
Test Error: 
 Accuracy: 48.1%, Avg loss: 0.032131 

Epoch 3
-------------------------------
loss: 2.017087  [    0/60000]
loss: 2.050029  [ 6400/60000]
loss: 1.971771  [12800/60000]
loss: 2.041152  [19200/60000]
loss: 1.918628  [25600/60000]
loss: 1.927792  [32000/600

## 4-保存模型

保存模型：通过序列化内部状态字典（包含模型参数）的方式。

函数：
`torch.save(model.state_dict(),"model.pth")`

In [9]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


## 5-加载模型和预测

In [10]:
model = NeuralNetwork()
model.load_state_dict(torch.load("data_ch01/model.pth"))

<All keys matched successfully>

模型预测


In [11]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    # pred是一个向量
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    # print
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"


总结：

+ 处理数据，得到dataLoader 

+ 创建模型

+ 定义（选择）损失函数，选择优化器，以优化模型参数

+ 保存模型。